## Lets check the customer data and create dim_customer table in gold layer

In [0]:
%sql

SELECT customer_key,
       customer_id,
       customer_name,
       email,
       phone,
       country,
       state,
       city
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY customer_id) AS rn,
           customer_id,
           customer_name,
           email,
           phone,
           country,
           state,
           city
    FROM salesdb.silver.crm_sales_data
    WHERE customer_id IS NOT NULL
) t
WHERE rn = 1


In [0]:
query =  """
SELECT customer_key,
       customer_id,
       customer_name,
       email,
       phone,
       country,
       state,
       city
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY customer_id) AS rn,
           customer_id,
           customer_name,
           email,
           phone,
           country,
           state,
           city
    FROM salesdb.silver.crm_sales_data
    WHERE customer_id IS NOT NULL
) t
WHERE rn = 1
 """
df_customer = spark.sql(query)
display(df_customer)
df_customer.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_customer")

In [0]:
%sql

select * from salesdb.silver.crm_sales_data

In [0]:
%sql

SELECT ROW_NUMBER() OVER (ORDER BY sales_rep_id) AS salesrep_key,  -- Surrogate key
       rn,
       sales_rep_id,
       sales_rep_name
FROM (
    SELECT 
           ROW_NUMBER() OVER (PARTITION BY sales_rep_id ORDER BY sales_rep_id) AS rn,
           sales_rep_id,
           sales_rep_name
    FROM salesdb.silver.crm_sales_data 
    WHERE sales_rep_id IS NOT NULL
) t



In [0]:
query =  """
SELECT ROW_NUMBER() OVER (ORDER BY sales_rep_id) AS salesrep_key,  -- Surrogate key
       sales_rep_id,
       sales_rep_name
FROM (
    SELECT 
           ROW_NUMBER() OVER (PARTITION BY sales_rep_id ORDER BY sales_rep_id) AS rn,
           sales_rep_id,
           sales_rep_name
    FROM salesdb.silver.crm_sales_data 
    WHERE sales_rep_id IS NOT NULL
) t
WHERE rn = 1
 """
df_salesrep_ = spark.sql(query)
display(df_salesrep_)
df_salesrep_.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_salesrep")



In [0]:
%sql
SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
      rn
       product_id,
       product_name,
       category
FROM (
    SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
           ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY product_id) AS rn,
           product_id,
           product_name,
           category
    FROM salesdb.silver.crm_sales_data 
    WHERE product_id IS NOT NULL
) t WHERE rn = 1


In [0]:
query =  """
SELECT ROW_NUMBER() OVER (ORDER BY product_id) AS product_key,  -- Surrogate key
       product_id,
       product_name,
       category
FROM (
    SELECT ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY product_id) AS rn,
           product_id,
           product_name,
           category
    FROM salesdb.silver.crm_sales_data 
    WHERE product_id IS NOT NULL
) t
WHERE rn = 1
 """


df_product = spark.sql(query)
display(df_product)
df_product.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_product")

In [0]:
%sql

-- Get start and end date dynamically from your sales data
WITH date_range AS (
    SELECT
        MIN(order_date) AS start_date,
        MAX(order_date) AS end_date
    FROM salesdb.silver.crm_sales_data
),
date_sequence AS (
    SELECT
        EXPLODE(SEQUENCE(start_date, end_date, INTERVAL 1 DAY)) AS date_key
    FROM date_range
)

-- Build Dim_Date with year, quarter, month
SELECT
    date_key,
    YEAR(date_key) AS year,
    QUARTER(date_key) AS quarter,
    MONTH(date_key) AS month
FROM date_sequence;


In [0]:
query =  """
WITH date_range AS (
    SELECT
        MIN(order_date) AS start_date,
        MAX(order_date) AS end_date
    FROM salesdb.silver.crm_sales_data
),
date_sequence AS (
    SELECT
        EXPLODE(SEQUENCE(start_date, end_date, INTERVAL 1 DAY)) AS date_key
    FROM date_range
)

-- Build Dim_Date with year, quarter, month
SELECT
    date_key,
    YEAR(date_key) AS year,
    QUARTER(date_key) AS quarter,
    MONTH(date_key) AS month
FROM date_sequence;
 """


df_dim_date = spark.sql(query)
display(df_dim_date)
df_dim_date.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.dim_date")

In [0]:
%sql

SELECT
    sd.order_id,
    sd.order_date,
    cust.customer_key,
    pr.product_key,
    srep.salesrep_key,
    d.date_key,
    SUM(sd.price * sd.quantity) AS total_sales,
    SUM(sd.quantity) AS total_quantity
FROM salesdb.silver.crm_sales_data AS sd
LEFT JOIN salesdb.gold.dim_customer AS cust ON sd.customer_id = cust.customer_id
LEFT JOIN salesdb.gold.dim_product AS pr ON sd.product_id = pr.product_id
LEFT JOIN salesdb.gold.dim_salesrep AS srep ON sd.sales_rep_id = srep.sales_rep_id
LEFT JOIN salesdb.gold.dim_date AS d ON sd.order_date = d.date_key
GROUP BY sd.order_id, sd.order_date, cust.customer_key, pr.product_key, srep.salesrep_key, d.date_key;


--drop table salesdb.gold.fact_sales

In [0]:
query =  """

SELECT
    sd.order_id,
    sd.order_date,
    cust.customer_key,
    pr.product_key,
    srep.salesrep_key,
    d.date_key,
    SUM(sd.price * sd.quantity) AS total_sales,
    SUM(sd.quantity) AS total_quantity
FROM salesdb.silver.crm_sales_data AS sd
LEFT JOIN salesdb.gold.dim_customer AS cust ON sd.customer_id = cust.customer_id
LEFT JOIN salesdb.gold.dim_product AS pr ON sd.product_id = pr.product_id
LEFT JOIN salesdb.gold.dim_salesrep AS srep ON sd.sales_rep_id = srep.sales_rep_id
LEFT JOIN salesdb.gold.dim_date AS d ON sd.order_date = d.date_key
GROUP BY sd.order_id, sd.order_date, cust.customer_key, pr.product_key, srep.salesrep_key, d.date_key;

 """


df_fact_sales = spark.sql(query)
display(df_fact_sales)
df_fact_sales.write.mode("overwrite").format("delta").saveAsTable("salesdb.gold.fact_sales")

In [0]:
%sql

CREATE OR REPLACE VIEW salesdb.gold.vw_sales_analysis AS
SELECT
    f.order_id,
    f.order_date,
    c.customer_name,
    p.product_name,
    s.sales_rep_name,
    d.year,
    d.month,
    f.total_sales,
    f.total_quantity
FROM salesdb.gold.fact_sales AS f
LEFT JOIN salesdb.gold.dim_customer AS c ON f.customer_key = c.customer_key
LEFT JOIN salesdb.gold.dim_product AS p ON f.product_key = p.product_key
LEFT JOIN salesdb.gold.dim_salesrep AS s ON f.salesrep_key = s.salesrep_key
LEFT JOIN salesdb.gold.dim_date AS d ON f.date_key = d.date_key;



In [0]:
%sql

SELECT
* from salesdb.gold.vw_sales_analysis